In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")

In [3]:
os.getcwd()

'd:\\Scripting\\DEEPCNNCLASSIFIER'

In [4]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig():
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size:list
    params_batch_size:int

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml
from distutils.command.config import config
from deepClassifier.utils.common import create_directories, save_json
from deepClassifier.constants import *

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            mlflow_uri = "https://dagshub.com/Hussain044/DEEPCNNCLASSIFIER.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
            )    
        return evaluation_config

In [9]:
import tensorflow as tf
import time
from pathlib import Path
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse

class Evaluation:
    def __init__(self,config=EvaluationConfig):
        self.config=config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        validation_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate(self):
        model = self.load_model(path=self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.validation_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}    
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_typr_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy":self.score[1]}
            )

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluate()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
313/313 [==============================] - 895s 3s/step - loss: 2.9978 - accuracy: 0.7917


EnsureError: Argument path of type <class 'str'> to <function save_json at 0x0000024BC567D3A0> does not match annotation type <class 'pathlib.Path'>